---

# 🏷️ Annotated Constraints

### 🎯 Intent

Use **`typing.Annotated`** with Pydantic constraints to make field rules **cleaner, modular, and reusable**.

---

### 🧩 Core Components

1. **📝 Basic Usage**

   * Attach constraints directly inside `Annotated`.
   * Example: `age: Annotated[int, Field(ge=18, le=99)]`.

2. **⚡ Multiple Constraints**

   * Stack multiple rules inside `Annotated`.
   * Example: `Annotated[str, Field(min_length=2, max_length=50)]`.

3. **📦 Reusable Types**

   * Define custom constrained types once, reuse everywhere.
   * Example: `PositiveInt = Annotated[int, Field(gt=0)]`.

4. **🔄 Works with TypeAdapter**

   * `TypeAdapter` accepts `Annotated` types for standalone validation.

5. **🌍 Benefits**

   * Cleaner than repeating `Field(...)`.
   * Great for consistency in shared schemas or libraries.

---


In [1]:
from typing import Annotated
from pydantic import BaseModel, Field, TypeAdapter, ValidationError

# 1. 📝 Reusable constrained types
PositiveInt = Annotated[int, Field(gt=0)]
Username = Annotated[str, Field(min_length=3, max_length=20, pattern=r"^[a-zA-Z0-9_]+$")]

# 2. 🧱 Model using Annotated fields
class User(BaseModel):
    id: PositiveInt                # must be > 0
    name: Username                 # min 3 chars, max 20, alphanumeric + underscore
    age: Annotated[int, Field(ge=18, le=99)]  # must be between 18–99

# ✅ Valid user
u = User(id=1, name="Mukesh_Y", age=25)
print("Valid user:", u.model_dump())

# ❌ Invalid user (age too low, name too short)
try:
    User(id=-1, name="x", age=15)
except ValidationError as e:
    print("\nValidation errors:", e.errors())

# 3. 🔄 Works with TypeAdapter
adapter = TypeAdapter(Username)
print("\nAdapter valid:", adapter.validate_python("Cool_User"))

try:
    adapter.validate_python("x!")  # invalid
except ValidationError as e:
    print("Adapter error:", e.errors())


Valid user: {'id': 1, 'name': 'Mukesh_Y', 'age': 25}

Validation errors: [{'type': 'greater_than', 'loc': ('id',), 'msg': 'Input should be greater than 0', 'input': -1, 'ctx': {'gt': 0}, 'url': 'https://errors.pydantic.dev/2.11/v/greater_than'}, {'type': 'string_too_short', 'loc': ('name',), 'msg': 'String should have at least 3 characters', 'input': 'x', 'ctx': {'min_length': 3}, 'url': 'https://errors.pydantic.dev/2.11/v/string_too_short'}, {'type': 'greater_than_equal', 'loc': ('age',), 'msg': 'Input should be greater than or equal to 18', 'input': 15, 'ctx': {'ge': 18}, 'url': 'https://errors.pydantic.dev/2.11/v/greater_than_equal'}]

Adapter valid: Cool_User
Adapter error: [{'type': 'string_too_short', 'loc': (), 'msg': 'String should have at least 3 characters', 'input': 'x!', 'ctx': {'min_length': 3}, 'url': 'https://errors.pydantic.dev/2.11/v/string_too_short'}]
